In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = '/content/drive/MyDrive/DATA/train'
validation_dir = '/content/drive/MyDrive/DATA/validation'

In [ ]:
num_classes = 92
image_size = (299,299)
batch_size = 64  ## 32
learning_rate = 0.001
epochs = 5

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [ ]:
valid_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
base_model = InceptionV3(weights = 'imagenet', include_top =False, input_shape = (image_size[0], image_size[1],3))

In [ ]:
x  = base_model.output
x = GlobalAveragePooling2D()(x)
x  = Dense(1024, activation='relu')(x)
prediction = Dense(num_classes, activation = 'softmax')(x)


In [ ]:
model = Model(inputs = base_model.input, outputs = prediction)

In [ ]:
for layer in base_model.layers:
  layer.trainable =False


In [ ]:
model.compile(optimizer = Adam(learning_rate = learning_rate), loss='categorical_crossentropy',metrics = ['accuracy'] )

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    validation_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 4355 images belonging to 92 classes.
Found 1094 images belonging to 92 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // batch_size
)

Epoch 1/5
68/68 [==============================] - 1487s 22s/step - loss: 0.1160 - accuracy: 0.9685 - val_loss: 0.5912 - val_accuracy: 0.8392
Epoch 2/5
68/68 [==============================] - 1445s 21s/step - loss: 0.1980 - accuracy: 0.9464 - val_loss: 0.3694 - val_accuracy: 0.8915
Epoch 3/5
36/68 [==============>...............] - ETA: 9:09 - loss: 0.0836 - accuracy: 0.9774

In [ ]:
model.save('/content/drive/MyDrive/inceptionV3_model.h5')